# Preprocessing

In [3]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 505 kB/s eta 0:00:01


In [4]:
!pip install toad

     |████████████████████████████████| 14.2 MB 1.1 MB/s eta 0:00:01


In [5]:
# import packages
import pandas as pd
import numpy as np
from category_encoders.count import CountEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb

from tqdm import tqdm
from tqdm import tqdm_notebook

import gc

import warnings
warnings.filterwarnings('ignore')

import toad
import matplotlib.pyplot as plt

In [6]:
pd.set_option('display.max_rows',None)

In [7]:
train_label = pd.read_csv('train_label_new.csv', index_col=0)
train_base = pd.read_csv('train_base_new.csv', index_col=0)
train_trans = pd.read_csv('train_trans_new.csv', index_col=0)
train_op = pd.read_csv('train_op_new.csv')
train_op = train_op.drop(columns=['Unnamed: 0'])

test_base = pd.read_csv('test_base_new.csv', index_col=0)
test_op = pd.read_csv('test_op_new.csv', index_col=0)
test_trans = pd.read_csv('test_trans_new.csv', index_col=0)
# test_label never show up to get rid of leakage problem!

In [8]:
pd.set_option('display.max_columns', 11)
train_base.head()

,user,sex,age,provider,level,...,product4_amount,product5_amount,product6_amount,product7_cnt,product7_fail_cnt
0,Train_06800,category 1,24877,category 0,category 2,...,level 0,level 0,level 1,24706,24706
1,Train_23487,category 1,24895,category 0,category 2,...,level 0,level 0,level 1,24712,24706
2,Train_36880,category 0,24853,category 0,category 2,...,level 0,level 0,level 1,24712,24706
3,Train_35392,category 0,24938,category 1,category 1,...,level 0,level 0,level 1,24712,24706
4,Train_35057,category 0,24956,category 0,category 2,...,level 0,level 0,level 1,24712,24706


In [9]:
pd.set_option('display.max_columns', 8)
train_trans.head()

,user,platform,tunnel_in,tunnel_out,...,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,...,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,...,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,...,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,...,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,...,NaN,NaN,NaN,26 days 07:50:05.000000000


In [8]:
# concatenate base and label
train_df = train_base.copy()
test_df = test_base.copy()
train_df = train_label.merge(train_df, on=['user'], how='left')
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
# del train_base, test_base

# concatenate train and test dataset of operation and transaction
op_df = pd.concat([train_op, test_op], axis=0, ignore_index=True)
trans_df = pd.concat([train_trans, test_trans], axis=0, ignore_index=True)
# del train_op, test_op, train_df, test_df, train_trans, test_trans

# Feature Engineering 1

In [9]:
# base = toad.detector.detect(data)
# base

In [10]:
# Since this feature have too many null values, so we drop this feature
data.drop(['service3_level'], axis=1, inplace=True)

In [11]:
# Transfer into integer
for col in ['balance', 'balance_avg', 'balance1', 'balance1_avg', 'balance2','balance2_avg', 'product1_amount', 'product2_amount',
          'product3_amount', 'product4_amount', 'product5_amount', 'product6_amount']:
    data[col] = data[col].apply(lambda x: int(x.split(' ')[1]) if type(x) != float else np.NaN)

In [12]:
original_cate = data.select_dtypes('object').columns

In [13]:
# Interactive item
cate_features = ['sex', 'provider', 'level', 'verified', 'regist_type', 'agreement1', 'agreement2', 'agreement3', 'agreement4', 
                 'province', 'city', 'service3']

for f1 in tqdm(cate_features):
    for f2 in cate_features:
        data['{}_{}'.format(f1, f2)] = data[f1] + '_' + data[f2]
        
for f in tqdm(cate_features):
    data['{}_cnt'.format(f)] = data.groupby([f])['user'].transform('count')

100%|██████████| 12/12 [00:00<00:00, 165.69it/s]


In [14]:
# data.dtypes

In [15]:
# using Label Encoder to deal with categorical features
for col in tqdm([col for col in original_cate  if col not in ['user']]):
    le = LabelEncoder()
    data[col].fillna('-1', inplace=True)
    data[col] = le.fit_transform(data[col])

100%|██████████| 12/12 [00:01<00:00, 10.60it/s]


In [16]:
# using Count Encoder to categorical features
train_base = pd.read_csv('train_base_new.csv')
test_base = pd.read_csv('test_base_new.csv')

df = pd.concat([train_base,test_base], axis=0, ignore_index=True)
# del train_base, test_base
df_category=df.select_dtypes('object')

df_category_nunique = df_category.nunique()
A_cnt_features = [col for col in df_category_nunique.index if df_category_nunique.loc[col] > 5 and col!='user']

frequency_fea = pd.DataFrame()
frequency_fea['user'] = df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    df_category[col] = df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = df_category[col].map(df_category[col].value_counts())
    
data=data.merge(frequency_fea,on="user",how="left")
# del df,df_category,df_category_nunique

In [17]:
dense_features = ['age', 'using_time', 'card_a_cnt', 'card_b_cnt', 'card_c_cnt', 'card_d_cnt', 'op1_cnt', 'op2_cnt', 'service1_cnt', 'service1_amt', 'service2_cnt', 
                  'agreement_total', 'acc_count', 'login_cnt_period1', 'login_cnt_period2', 'ip_cnt', 'login_cnt_avg', 'login_days_cnt', 'balance', 'balance_avg', 
                  'balance1', 'balance1_avg', 'balance2', 'balance2_avg', 'product1_amount', 'product2_amount', 'product3_amount', 'product4_amount', 'product5_amount',
                 'product6_amount', 'product7_cnt', 'product7_fail_cnt']

min_max = MinMaxScaler()
data[dense_features] = min_max.fit_transform(data[dense_features].values)

In [18]:
# generate features from experience
data['product7_fail_ratio'] = data['product7_fail_cnt'] / data['product7_cnt']
data['city_count'] = data.groupby(['city'])['user'].transform('count')
data['province_count'] = data.groupby(['province'])['user'].transform('count')

data['card_cnt'] = data['card_a_cnt'] + data['card_b_cnt'] + data['card_c_cnt'] + data['card_d_cnt']

data['acc_card_ratio'] = data['acc_count'] / data['card_cnt']
data['login_cnt'] = data['login_cnt_period1'] + data['login_cnt_period2']

data['login_cnt_period2_login_cnt_ratio'] = data['login_cnt_period2'] / data['login_cnt']
data['login_cnt_period1_login_cnt_ratio'] = data['login_cnt_period1'] / data['login_cnt']

data['using_time_op2_cnt_ratio'] = data['using_time'] / data['op2_cnt']
data['using_time_op1_cnt_ratio'] = data['using_time'] / data['op1_cnt']

In [19]:
data.shape

(47782, 223)

In [20]:
data.to_csv('feature_base.csv', index=False)

Feature Engineering on trans.csv

In [21]:
#transform variable tm_diff
def transform_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400*day+3600*hour+60*minute+second

# extract elements from tm_diff
# timestamp means how many seconds passed after the start point
op_df['day'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
trans_df['day'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
op_df['timestamp'] = op_df['tm_diff'].apply(lambda x: transform_time(x))
trans_df['timestamp'] = trans_df['tm_diff'].apply(lambda x: transform_time(x))
op_df['hour'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
trans_df['hour'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
trans_df['week'] = trans_df['day'].apply(lambda x: x % 7)
op_df['min'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[1]))
op_df['second'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[2]))

# using timestamp to reorganize the sequence of each user's behaviour
trans_df = trans_df.sort_values(by=['user', 'timestamp'])
op_df = op_df.sort_values(by=['user', 'timestamp'])
trans_df.reset_index(inplace=True, drop=True)
op_df.reset_index(inplace=True, drop=True)

gc.collect()

15

In [22]:
# use hour to group the behaviours in trans and op
trans_df["time"]="time"
trans_df.loc[trans_df.hour<=6,"time"]="night"
trans_df.loc[(trans_df.hour>6)&(trans_df.hour<=12),"time"]="morning"
trans_df.loc[(trans_df.hour>12)&(trans_df.hour<=18),"time"]="afternoon"
trans_df.loc[trans_df.hour>18,"time"]="evening"

op_df["time"]="time"
op_df.loc[op_df.hour<=6,"time"]="night"
op_df.loc[(op_df.hour>6)&(op_df.hour<=12),"time"]="morning"
op_df.loc[(op_df.hour>12)&(op_df.hour<=18),"time"]="afternoon"
op_df.loc[op_df.hour>18,"time"]="evening"

Features about transcation amount

In [23]:
# define a function that calculate statistics about transaction amount
def gen_user_amount_features(df):
    group_df = df.groupby(['user'])['amount'].agg([
        ('user_amount_mean', 'mean'),
        ('user_amount_std','std'),
        ('user_amount_max', 'max'),
        ('user_amount_min', 'min'),
        ('user_amount_sum', 'sum'),
        ('user_amount_med', 'median'),
        ('user_amount_cnt', 'count')
    ]).reset_index()
    return group_df

# extract features about transcation amount
data = data.merge(gen_user_amount_features(trans_df), on=['user'], how='left')

In [24]:
# define unique value of each user
def gen_user_nunique_features(df, value, prefix):
    group_df = df.groupby(['user'])[value].agg([
        ('user_{}_{}_nuniq'.format(prefix, value), 'nunique')]
    ).reset_index()
    return group_df

# extract features from trans dataframe
for col in tqdm(['day', 'platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2', 'ip', 'ip_3']):
    data = data.merge(gen_user_nunique_features(df=trans_df, value=col, prefix='trans'), on=['user'], how='left')
    
# transaction amount per day
data['user_amount_per_days'] = data['user_amount_sum'] / data['user_trans_day_nuniq']
# transcation amount each time
data['user_amount_per_cnt'] = data['user_amount_sum'] / data['user_amount_cnt']

100%|██████████| 8/8 [00:02<00:00,  2.98it/s]


In [25]:
# define a function to get the transcation amount with each group
def gen_user_group_amount_features(df, value):
    group_df = df.pivot_table(index='user',
                              columns=value,
                              values='amount',
                              dropna=False,
                              aggfunc=['count', 'sum'])
    group_df.columns = ['user_{}_{}_amount_{}'.format(value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)

    return group_df

# group by platform
data = data.merge(gen_user_group_amount_features(df=trans_df, value='platform'), on=['user'], how='left')
# gourp by type1
data = data.merge(gen_user_group_amount_features(df=trans_df, value='type1'), on=['user'], how='left')
# group by type2
data = data.merge(gen_user_group_amount_features(df=trans_df, value='type2'), on=['user'], how='left')
# group by time
data = data.merge(gen_user_group_amount_features(df=trans_df, value='time'), on=['user'], how='left')
#group by week
data = data.merge(gen_user_group_amount_features(df=trans_df, value='week'), on=['user'], how='left')

In [26]:
# add time axis, define a function to calculate users' transcation amount linked with feature 'days'
def gen_user_window_amount_features(df, window):
    group_df = df[df['day']>window].groupby('user')['amount'].agg([
        ('user_amount_mean_{}d'.format(window), 'mean'),
        ('user_amount_std_{}d'.format(window),'std'),
        ('user_amount_max_{}d'.format(window),'max'),
        ('user_amount_min_{}d'.format(window), 'min'),
        ('user_amount_sum_{}d'.format(window),'sum'),
        ('user_amount_med_{}d'.format(window),'median'),
        ('user_amount_cnt_{}d'.format(window),'count')
    ]).reset_index()
    return group_df

# extract amount feature within transcations after 7 days
data = data.merge(gen_user_window_amount_features(df=trans_df, window=7), on=['user'], how='left')
# extract amount feature within transcations after 14 days
data = data.merge(gen_user_window_amount_features(df=trans_df, window=14), on=['user'], how='left')
# extract amount feature within transcations after 21 days
data = data.merge(gen_user_window_amount_features(df=trans_df, window=21), on=['user'], how='left')
# extract amount feature within transcations after 28 days
data = data.merge(gen_user_window_amount_features(df=trans_df, window=28), on=['user'], how='left')

In [27]:
# define a function to calculate users' transcation amount linked with feature 'hours'
def gen_user_window_amount_features(df, window):
    group_df = df[df['hour']>window].groupby('user')['amount'].agg([
        ('user_amount_mean_{}h'.format(window), 'mean'),
        ('user_amount_std_{}h'.format(window),'std'),
        ('user_amount_max_{}h'.format(window),'max'),
        ('user_amount_min_{}h'.format(window),'min'),
        ('user_amount_sum_{}h'.format(window), 'sum'),
        ('user_amount_med_{}h'.format(window), 'median'),
        ('user_amount_cnt_{}h'.format(window), 'count')
    ]).reset_index()
    return group_df

# extract amount feature within transcations after 6 a.m.
data = data.merge(gen_user_window_amount_features(df=trans_df, window=6), on=['user'], how='left')
# extract amount feature within transcations after 12 p.m.
data = data.merge(gen_user_window_amount_features(df=trans_df, window=12), on=['user'], how='left')
# extract amount feature within transcations after 18 p.m.
data = data.merge(gen_user_window_amount_features(df=trans_df, window=18), on=['user'], how='left')

In [28]:
# for some features, we consider that null comes from some reason, so we generate features about missing value
def gen_user_null_features(df, value, prefix):
    df['is_null'] = 0
    df.loc[df[value].isnull(), 'is_null'] = 1

    group_df = df.groupby(['user'])['is_null'].agg([('user_{}_{}_null_cnt'.format(prefix, value), 'sum'),
                                                    ('user_{}_{}_null_ratio'.format(prefix, value),'mean')]).reset_index()
    return group_df

# extract missing value and ratio within ip address
data = data.merge(gen_user_null_features(df=trans_df, value='ip', prefix='trans'), on=['user'], how='left')

In [29]:
# extract the first apperance of "type1 == 45a1168437c708ff"
group_df = trans_df[trans_df['type1']=='45a1168437c708ff'].groupby(['user'])['day'].agg([('user_type1_45a1168437c708ff_min_day', 'min')]).reset_index()
data = data.merge(group_df, on=['user'], how='left')
del group_df

In [30]:
# define the transaction amount of each users per day and hour
def per_hour_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('per_hour_amt_sum','sum'),
        ('per_hour_amt_mean','mean'),
        ('per_hour_amt_max','max'),
        ('per_hour_amt_min','min'),
        ('per_hour_amt_cnt','count'),
        ('per_hour_amt_std','std')
    ]).reset_index()
    return group_df

# extract transcation per day and hour features
data = data.merge(per_hour_amt(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [31]:
# define a function to calculate each user's transcation amount group by tunnel_in and tunnel_out
def tunnel_in_out_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('tunnel_in_out_amt_sum','sum'),
        ('tunnel_in_out_amt_cnt','count'),
        ('tunnel_in_out_amt_mean','mean'),
        ('tunnel_in_out_amt_max','max'),
        ('tunnel_in_out_amt_min','min'),
        ('tunnel_in_out_amt_std','std'),
    ]).reset_index()
    return group_df

# extract trascation amount group by tunnel_in and tunnel_out
data = data.merge(tunnel_in_out_amt(trans_df,value1="tunnel_in",value2="tunnel_out"), on=['user'], how='left')

In [32]:
# define a function to calculate each user's transcation amount group by "ip"
def day_ip_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_ip_amt_sum','sum'),
        ('day_ip_amt_mean','mean'),
        ('day_ip_amt_std','std'),
        ('day_ip_amt_max','max'),
        ('day_ip_amt_min','min'),
        ('day_ip_amt_cnt','count')
    ]).reset_index()
    return group_df

# extract trascation amount group by "ip"
data = data.merge(day_ip_amt(trans_df,value1="day",value2="ip"), on=['user'], how='left')

In [33]:
# define a function to find the gap between transcations of each user for each hour
def amt_gap(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['amount_gap']=abs(group_df["amount"]-group_df["last_amount"])
    group_df=group_df[['user','amount_gap']]
    group_df=group_df.groupby('user')['amount_gap'].agg([
        ('amt_gap_sum','sum'),
        ('amt_gap_mean','mean'),
        ('amt_gap_std','std'),
        ('amt_gap_max','max'),
        ('amt_gap_min','min')
    ]).reset_index()
    return group_df

# extract features from the amount of transcation group by time gap
data = data.merge(amt_gap(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [34]:
# define a function to calculate the sum of gap ratio of amount for each user
def gap_amt_rate(df,value1,value2):
    group_df=df[['user',value1,value2,'amount']]
    group_df=group_df.groupby(['user', 'day', 'hour'])['amount'].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['gap_rate']=group_df['amount']/group_df['last_amount']
    group_df=group_df[['user','gap_rate']]
    group_df=group_df.groupby('user')['gap_rate'].agg('sum').reset_index()
    return group_df

# extract the gap ratio feature
data = data.merge(gap_amt_rate(trans_df,value1="day",value2="hour"), on=['user'], how='left')

In [35]:
# generate the amount of transaction per day
def per_day_amt(df,value):
    group_df=df[['user',value, 'amount']]
    group_df=group_df.groupby(['user',value])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('per_day_amt_mean','mean'),
        ('per_day_amt_max','max'),
        ('per_day_amt_min','min'),
        ('per_day_amt_std','std')
    ]).reset_index()
    return group_df

# extract features using the function
data = data.merge(per_day_amt(trans_df,value="day"), on=['user'], how='left')

In [36]:
# define a function to find the gap between transcations of each user for each transcation day
def day_amt_gap(df,value):
    group_df=df[['user',value, 'amount']]
    group_df=group_df.groupby(['user',value])["amount"].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['amount_gap']=abs(group_df["amount"]-group_df["last_amount"])
    group_df=group_df[['user','amount_gap']]
    group_df=group_df.groupby('user')['amount_gap'].agg([
        ('day_amt_gap_mean','mean'),
        ('day_amt_gap_std','std'),
        ('day_amt_gap_max','max'),
        ('day_amt_gap_min','min')
    ]).reset_index()
    return group_df

# extract amount gap per day
data = data.merge(day_amt_gap(trans_df,value="day"), on=['user'], how='left')

In [37]:
# define a function to calculate the gap ratio per day
def day_gap_amt_rate(df,value):
    group_df=df[['user',value,'amount']]
    group_df=group_df.groupby(['user', value])['amount'].agg('sum').reset_index()
    group_df['last_amount']=group_df.groupby('user')['amount'].shift(1)
    group_df['day_gap_rate']=group_df['amount']/group_df['last_amount']
    group_df=group_df[['user','day_gap_rate']]
    group_df=group_df.groupby('user')['day_gap_rate'].agg('sum').reset_index()
    return group_df

# generate amount gap ratio per day
data = data.merge(day_gap_amt_rate(trans_df,value="day"), on=['user'], how='left')

In [38]:
# define a function to calculate the type1 transcation per day
def day_type_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_type1_amt_sum','sum'),
        ('day_type1_amt_mean','mean'),
        ('day_type1_amt_std','std'),
        ('day_type1_amt_max','max'),
        ('day_type1_amt_min','min'),
        ('day_type1_amt_cnt','count')
    ]).reset_index()
    return group_df

# extract type1 transcation amount per day
data = data.merge(day_type_amt(trans_df,value1="day",value2="type1"), on=['user'], how='left')

In [39]:
# define a function to calculate the type2 transcation per day
def day_type2_amt(df,value1,value2):
    group_df=df[['user',value1,value2, 'amount']]
    group_df=group_df.groupby(['user',value1,value2])["amount"].agg('sum').reset_index()
    group_df=group_df[['user','amount']]
    group_df=group_df.groupby('user')['amount'].agg([
        ('day_type2_amt_sum','sum'),
        ('day_type2_amt_mean','mean'),
        ('day_type2_amt_std','std'),
        ('day_type2_amt_max','max'),
        ('day_type2_amt_min','min'),
        ('day_type2_amt_cnt','count')
    ]).reset_index()
    return group_df

# extract type2 transcation amount per day
data = data.merge(day_type2_amt(trans_df,value1="day",value2="type2"), on=['user'], how='left')

In [40]:
data.head()

,user,label,sex,age,...,day_type2_amt_std,day_type2_amt_max,day_type2_amt_min,day_type2_amt_cnt
0,Train_00000,0.0,1,0.227586,...,129348.034147,321635.0,54093.0,4.0
1,Train_00001,1.0,1,0.252414,...,NaN,36098.0,36098.0,1.0
2,Train_00002,0.0,1,0.354483,...,30670.951279,221240.0,162423.0,3.0
3,Train_00005,0.0,1,0.176552,...,70341.901890,267494.0,39630.0,12.0
4,Train_00006,0.0,1,0.219310,...,90202.285063,339168.0,34914.0,9.0


Word2Vec

In [41]:
from gensim.models import Word2Vec
import multiprocessing
def w2v_feat(data_frame, feat, mode):
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=5, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_trans_{t}_{n}' for n in range(5)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all

# generate word2vec features
trans_feat=["platform","tunnel_in","tunnel_out","amount","type1","type2","ip","day","hour"]
data=data.merge(w2v_feat(trans_df,trans_feat,'trans'), on=['user'], how='left')

Start trans word2vec ...
Start gen feat of platform ...
Start gen feat of tunnel_in ...
Start gen feat of tunnel_out ...
Start gen feat of amount ...
Start gen feat of type1 ...
Start gen feat of type2 ...
Start gen feat of ip ...
Start gen feat of day ...
Start gen feat of hour ...


In [42]:
data.shape

(47782, 615)

In [43]:
# using Count Encoder to categorical features in df_trans
from category_encoders.count import CountEncoder
for i in ["ip","ip_3","amount"]:
    trans_df["count_{}_trans".format(i)]=CountEncoder().fit_transform(trans_df[i])
    group_df=trans_df.groupby('user')["count_{}_trans".format(i)].agg([
        ("count_{}_trans_max".format(i),'max'),
        ("count_{}_trans_min".format(i), 'min'),
        ("count_{}_trans_mean".format(i),'mean'),
        ("count_{}_trans_std".format(i),'std')
    ]).reset_index()
    data=data.merge(group_df,on="user",how="left")
    
trans_df.drop(columns=trans_df.columns[-3:],inplace=True)

# extract frequncy features for features which have more than five unique values
trans_df_category=trans_df.drop(columns=["tm_diff","time"])
trans_df_category_nunique = trans_df_category.nunique()
A_cnt_features = [col for col in trans_df_category_nunique.index if trans_df_category_nunique.loc[col] > 5 and col!='user']
# print(len(A_cnt_features))
frequency_fea = pd.DataFrame()
frequency_fea['user'] = trans_df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    trans_df_category[col] = trans_df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = trans_df_category[col].map(trans_df_category[col].value_counts())
    
for i in tqdm_notebook(frequency_fea.columns[1:]):
    group_df=frequency_fea[["user",i]]
    group_df=group_df.groupby("user")[i].agg([
        ('freq_{}_max'.format(i),'max'),
        ('freq_{}_min'.format(i),'min'),
        ('freq_{}_mean'.format(i),'mean'),
        ('freq_{}_std'.format(i),'std')]).reset_index()
    data=data.merge(group_df,on="user",how="left")

In [44]:
data.shape

(47782, 671)

Extract features from op.csv

In [45]:
op_df.head()

,user,op_type,op_mode,op_device,...,hour,min,second,time
0,Train_00000,b26bc49195bd79cf,87ee0bdf333a54da,4ff62c2e280249dc,...,13,4,34,afternoon
1,Train_00000,b26bc49195bd79cf,6b0823f22acf82cf,4ff62c2e280249dc,...,13,4,45,afternoon
2,Train_00000,3089f3f91de53eee,3089f3f91de53eee,4ff62c2e280249dc,...,13,4,45,afternoon
3,Train_00000,b26bc49195bd79cf,6b0823f22acf82cf,4ff62c2e280249dc,...,13,4,55,afternoon
4,Train_00000,3089f3f91de53eee,3089f3f91de53eee,4ff62c2e280249dc,...,13,4,55,afternoon


In [46]:
def gen_user_tfidf_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    # use SVD method to reduce the dimension of this sparse matrix
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=623)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_tfidf_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

data = data.merge(gen_user_tfidf_features(df=op_df, value='op_mode'), on=['user'], how='left')
data = data.merge(gen_user_tfidf_features(df=op_df, value='op_type'), on=['user'], how='left')

In [47]:
def gen_user_countvec_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=623)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

data = data.merge(gen_user_countvec_features(df=op_df, value='op_mode'), on=['user'], how='left')
data = data.merge(gen_user_countvec_features(df=op_df, value='op_type'), on=['user'], how='left')

In [48]:
from gensim.models import Word2Vec
import multiprocessing
def w2v_feat(data_frame, feat, mode):
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=5, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_op_{t}_{n}' for n in range(5)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all

#生成word2vec特征
op_feat=["op_type","op_mode","op_device","ip","channel","day","hour"]
data=data.merge(w2v_feat(op_df,op_feat,'op'), on=['user'], how='left')

Start op word2vec ...
Start gen feat of op_type ...
Start gen feat of op_mode ...
Start gen feat of op_device ...
Start gen feat of ip ...
Start gen feat of channel ...
Start gen feat of day ...
Start gen feat of hour ...


In [49]:
# using Counter Encoder to generate features from op_df
from category_encoders.count import CountEncoder
for i in ["ip","ip_3","op_device","op_type","op_mode"]:
    op_df["count_{}_op".format(i)]=CountEncoder().fit_transform(op_df[i])
    group_df=op_df.groupby('user')["count_{}_op".format(i)].agg([
        ("count_{}_op_max".format(i),'max'),
        ("count_{}_op_min".format(i), 'min'),
        ("count_{}_op_mean".format(i),'mean'),
        ("count_{}_op_std".format(i),'std')
    ]).reset_index()
    data=data.merge(group_df,on="user",how="left")
    
op_df.drop(columns=op_df.columns[-5:],inplace=True)

In [50]:
# encode the frequency of opeartion and calculate statistics
op_df_category=op_df.drop(columns=["tm_diff","time"])
op_df_category_nunique = op_df_category.nunique()
A_cnt_features = [col for col in op_df_category_nunique.index if op_df_category_nunique.loc[col] > 5 and col!='user']
print(len(A_cnt_features))
frequency_fea = pd.DataFrame()
frequency_fea['user'] = op_df_category['user'].values
for col in tqdm_notebook(A_cnt_features):
    op_df_category[col] = op_df_category[col].fillna(-999)
    frequency_fea[col + '_cnt'] = op_df_category[col].map(op_df_category[col].value_counts())

for i in tqdm_notebook(frequency_fea.columns[1:]):
    group_df=frequency_fea[["user",i]]
    group_df=group_df.groupby("user")[i].agg([
       ('freq_{}_max'.format(i),'max'),
        ('freq_{}_min'.format(i),'min'),
        ('freq_{}_mean'.format(i),'mean'),
        ('freq_{}_std'.format(i),'std')]).reset_index()
    data=data.merge(group_df,on="user",how="left")

11


In [51]:
# define a function to calculate the possible values of each feature for each user
def gen_user_nunique_features(df, value, prefix):
    group_df = df.groupby(['user'])[value].agg([
        ('user_{}_{}_nuniq'.format(prefix, value),'nunique')
    ]).reset_index()
    return group_df

# extract number of unique values of each user
for col in tqdm(['op_type', 'op_mode', 'ip', 'channel', 'ip_3', 'day']):
    data = data.merge(gen_user_nunique_features(df=op_df, value=col, prefix='op'), on=['user'], how='left')

100%|██████████| 6/6 [00:05<00:00,  1.03it/s]


In [52]:
# define a function to count how many times of one operation type happens group by day
def gen_user_window_op_features(df, window):
    group_df = df[df['day']>window].groupby('user')['op_type'].agg([
        ('user_op_cnt_{}d'.format(window),'count')
    ]).reset_index()
    return group_df

op_df["day"]=op_df["day"].astype(int)

# extract operation type counting after 5 days
data = data.merge(gen_user_window_op_features(df=op_df, window=5), on=['user'], how='left')
# extract operation type counting after 10 days
data = data.merge(gen_user_window_op_features(df=op_df, window=10), on=['user'], how='left')

In [53]:
# define a function to count how many times of one operation type happens group by hour
def gen_op_window_hour_features(df, window):
    group_df = df[df['hour']>window].groupby('user')['op_type'].agg([
        ('user_op_cnt_{}h'.format(window),'count')]).reset_index()
    return group_df

op_df["hour"]=op_df["hour"].astype(int)
# extract operation type counting after 6
data = data.merge(gen_op_window_hour_features(df=op_df, window=6), on=['user'], how='left')
# extract operation type counting after 12
data = data.merge(gen_op_window_hour_features(df=op_df, window=12), on=['user'], how='left')
# extract operation type counting after 18
data = data.merge(gen_op_window_hour_features(df=op_df, window=18), on=['user'], how='left')

In [54]:
# define a function to count how many times of one operation type happens group by time(i.e morning, afternoon, evening and night)
def gen_user_group_op_features(df, value):
    group_df = df.pivot_table(index='user',
                              columns=value,
                              values='op_type',
                              dropna=False,
                              aggfunc=['count'])
    group_df.columns = ['user_{}_{}_op_{}'.format(value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)

    return group_df

# extrate operation type counts group by time
data = data.merge(gen_user_group_op_features(df=op_df, value='time'), on=['user'], how='left')

In [55]:
# counting how many times each user operated the app
op_count = op_df[['user']]
op_count['op_count'] = 1
op_count = op_count.groupby('user').agg('count').reset_index()
data = pd.merge(data, op_count, on='user', how='left')
# del op_count

# calculate how many times of operation happened per day
data["op_cnt_per_day"]=data["op_count"]/data["user_op_day_nuniq"]

In [56]:
# define a function to calculate the statistics of the counting of operations of each user in per hour per day
def day_per_hour_cnt(df,value1,value2):
    group_df = op_df[['user', 'day', 'hour']]
    group_df['everyday_everyhour'] = 1
    group_df = group_df.groupby(['user', 'day', 'hour']).agg('count').reset_index()
    group_df = group_df.drop(['day', 'hour'],axis = 1)
    group_df = group_df.groupby('user')['everyday_everyhour'].agg([
        ('day_per_hour_mean','mean'),
        ('day_per_hour_max','max'),
        ('day_per_hour_min','min'), 
        ('day_per_hour_std','std')]).reset_index()
    return group_df

data = data.merge(day_per_hour_cnt(op_df,"day","hour"), on='user', how='left')

In [57]:
# define a function to calculate the statistics of the counting of operations of each user in per day
frequence_one_day = op_df[['user', 'day']]
frequence_one_day['everyday'] = 1
frequence_one_day = frequence_one_day.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day = frequence_one_day.drop('day', axis=1)
frequence_one_day = frequence_one_day.groupby('user')['everyday'].agg([
    ('per_day_mean','mean'),
    ('per_day_max','max'),
    ('per_day_min','min'),
    ('per_day_std','std')]).reset_index()
data = data.merge(frequence_one_day, on='user', how='left')

In [58]:
# counting the opeartions happened in the morning for each user
frequence_morning = op_df[op_df.time=="morning"][['user', 'day','hour']]
frequence_morning['everyday_morning'] = 1
frequence_morning = frequence_morning.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_morning = frequence_morning.groupby(['user', 'day'])['everyday_morning'].agg('sum').reset_index()
frequence_morning = frequence_morning[['user', 'everyday_morning']]
frequence_morning = frequence_morning.groupby('user')['everyday_morning'].agg([
    ('per_mor_mean','mean'),
    ('per_mor_max','max'),
    ('per_mor_min','min'),
    ('per_mor_std','std')]).reset_index()
data = data.merge(frequence_morning, on='user', how='left')
del frequence_morning

In [59]:
# counting the opeartions happened in the afternoon for each user
frequence_afternoon = op_df[op_df.time=="afternoon"][['user', 'day','hour']]
frequence_afternoon['everyday_afternoon'] = 1
frequence_afternoon = frequence_afternoon.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_afternoon = frequence_afternoon.groupby(['user', 'day'])['everyday_afternoon'].agg('sum').reset_index()
frequence_afternoon = frequence_afternoon[['user', 'everyday_afternoon']]
frequence_afternoon = frequence_afternoon.groupby('user')['everyday_afternoon'].agg([
    ('per_after_mean','mean'),
    ('per_after_max','max'),
    ('per_after_min','min'),
    ('per_after_std','std')]).reset_index()
data = data.merge(frequence_afternoon, on='user', how='left')
del frequence_afternoon

In [60]:
# counting the opeartions happened in the evening for each user
frequence_evening = op_df[op_df.time=="evening"][['user', 'day','hour']]
frequence_evening['everyday_evening'] = 1
frequence_evening = frequence_evening.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_evening = frequence_evening.groupby(['user', 'day'])['everyday_evening'].agg('sum').reset_index()
frequence_evening = frequence_evening[['user', 'everyday_evening']]
frequence_evening = frequence_evening.groupby('user')['everyday_evening'].agg([
    ('per_eve_mean','mean'),
    ('per_eve_max','max'),
    ('per_eve_min','min'),
    ('per_eve_std','std')]).reset_index()
data = data.merge(frequence_evening, on='user', how='left')
del frequence_evening

In [61]:
# counting the opeartions happened at night for each user
frequence_night = op_df[op_df.time=="night"][['user', 'day','hour']]
frequence_night['everyday_night'] = 1
frequence_night = frequence_night.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_night = frequence_night.groupby(['user', 'day'])['everyday_night'].agg('sum').reset_index()
frequence_night = frequence_night[['user', 'everyday_night']]
frequence_night = frequence_night.groupby('user')['everyday_night'].agg([
    ('per_night_mean','mean'),
    ('per_night_max','max'),
    ('per_night_min','min'),
    ('per_night_std','std')]).reset_index()
data = data.merge(frequence_night, on='user', how='left')
del frequence_night

In [62]:
# calculate the statistics of the operation counting between two days
frequence_one_day_gap = op_df[['user', 'day']]
frequence_one_day_gap['everyday'] = 1
frequence_one_day_gap = frequence_one_day_gap.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day_gap['everyday_before'] = frequence_one_day_gap.groupby('user')['everyday'].shift(1)
frequence_one_day_gap['everyday_before_gap'] = abs(frequence_one_day_gap['everyday'] - frequence_one_day_gap['everyday_before'])
frequence_one_day_gap = frequence_one_day_gap[['user', 'everyday_before_gap']].groupby('user')['everyday_before_gap'].agg([
    ('op_day_gap_mean','mean'),
    ('op_day_gap_max','max'),
    ('op_day_gap_min','min'),
    ('op_day_gap_std','std')]).reset_index()
data = data.merge(frequence_one_day_gap, on='user', how='left')
del frequence_one_day_gap

In [63]:
# calculate the statistics of the operation counting between two hours
frequence_one_hour_gap = op_df[['user', 'day', 'hour']]
frequence_one_hour_gap['everyhour'] = 1
frequence_one_hour_gap = frequence_one_hour_gap.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_one_hour_gap['everyhour_before'] = frequence_one_hour_gap.groupby('user')['everyhour'].shift(1)
frequence_one_hour_gap['everyhour_before_gap'] = abs(frequence_one_hour_gap['everyhour'] - frequence_one_hour_gap['everyhour_before'])
frequence_one_hour_gap = frequence_one_hour_gap[['user', 'everyhour_before_gap']].groupby('user')['everyhour_before_gap'].agg([
    ('hour_gap_mean','mean'),
    ('hour_gap_max','max'),
    ('hour_gap_min','min'),
    ('hour_gap_std','std')]).reset_index()
data = data.merge(frequence_one_hour_gap, on='user', how='left')
del frequence_one_hour_gap

In [64]:
# calculate the ratio of the number of the operations between two days
frequence_one_day_rate = op_df[['user', 'day']]
frequence_one_day_rate['everyday'] = 1
frequence_one_day_rate = frequence_one_day_rate.groupby(['user', 'day']).agg('count').reset_index()
frequence_one_day_rate['everyday_before'] = frequence_one_day_rate.groupby('user')['everyday'].shift(1)
frequence_one_day_rate['everyday_before_rate'] = frequence_one_day_rate['everyday'] / frequence_one_day_rate['everyday_before']
frequence_one_day_rate = frequence_one_day_rate[['user', 'everyday_before_rate']].groupby('user')['everyday_before_rate'].agg('sum').reset_index()
data = data.merge(frequence_one_day_rate, on='user', how='left')
del frequence_one_day_rate

In [65]:
# calculate the ratio of the number of the operations between two hours
frequence_one_hour_rate = op_df[['user', 'day', 'hour']]
frequence_one_hour_rate['everyhour'] = 1
frequence_one_hour_rate = frequence_one_hour_rate.groupby(['user', 'day', 'hour']).agg('count').reset_index()
frequence_one_hour_rate['everyhour_before'] = frequence_one_hour_rate.groupby('user')['everyhour'].shift(1)
frequence_one_hour_rate['everyhour_before_rate'] = frequence_one_hour_rate['everyhour'] - frequence_one_hour_rate['everyhour_before']
frequence_one_hour_rate = frequence_one_hour_rate[['user', 'everyhour_before_rate']].groupby('user')['everyhour_before_rate'].agg('sum').reset_index()
data = data.merge(frequence_one_hour_rate, on='user', how='left')
del frequence_one_hour_rate

In [66]:
# calculate the statistics of the number of operatins of each user happened per second
every_second = op_df[['user', 'second']]
every_second['operation_second_op'] = 1
every_second = every_second.groupby(['user', 'second']).agg('count').reset_index()
every_second = every_second[['user', 'operation_second_op']]
every_second = every_second.groupby('user')['operation_second_op'].agg([
    ('per_sec_max','max'),
    ('per_sec_min','min'),
    ('per_sec_mean','mean'),
    ('per_sec_std','std')]).reset_index()
data = data.merge(every_second, on='user', how='left')
del every_second

In [67]:
# define a function to calculate the statistics of the number of operatins of each user happened per minutes
every_minute = op_df[['user', 'min']]
every_minute['operation_minute_op'] = 1
every_minute = every_minute.groupby(['user', 'min']).agg('count').reset_index()
every_minute = every_minute[['user', 'operation_minute_op']]
every_minute = every_minute.groupby('user')['operation_minute_op'].agg([
    ('per_minute_max','max'),
    ('per_minute_min','min'),
    ('per_minute_mean','mean'),
    ('per_minute_std','std')]).reset_index()
data = data.merge(every_minute, on='user', how='left')
del every_minute

In [68]:
# calculate the statistics of the number of different devices one may use per day
dev_per_day_cnt=op_df[op_df.op_device!="nan"][["user","day","op_device"]].drop_duplicates()
dev_per_day_cnt=pd.DataFrame(dev_per_day_cnt.groupby(["user","day"])["op_device"].nunique()).reset_index()
dev_per_day_cnt=dev_per_day_cnt[["user","op_device"]]
dev_per_day_cnt=dev_per_day_cnt.groupby(['user'])["op_device"].agg([
    ("dev_nun_mean","mean"),
    ("dev_nun_max","max"),
    ("dev_nun_min","min"),
    ("dev_nun_std","std")
]).reset_index()
data=data.merge(dev_per_day_cnt, on='user', how='left')
del dev_per_day_cnt

In [69]:
# calculate the statistics of the number of different devices one may use per hour
dev_per_hour_cnt=op_df[op_df.op_device!="nan"][["user","day","hour","op_device"]].drop_duplicates()
dev_per_hour_cnt=pd.DataFrame(dev_per_hour_cnt.groupby(["user","day","hour"])["op_device"].nunique()).reset_index()
dev_per_hour_cnt=dev_per_hour_cnt[["user","op_device"]]
dev_per_hour_cnt=dev_per_hour_cnt.groupby(['user'])["op_device"].agg([
    ("dev_hour_nun_mean","mean"),
    ("dev_hour_nun_max","max"),
    ("dev_hour_nun_min","min"),
    ("dev_hour_nun_std","std")
]).reset_index()
data=data.merge(dev_per_hour_cnt, on='user', how='left')
del dev_per_hour_cnt

In [70]:
# calculate the statistics of the number of different ip one may use per day
ip_per_day_cnt=op_df[op_df.ip!="nan"][["user","day","ip"]].drop_duplicates()
ip_per_day_cnt=pd.DataFrame(ip_per_day_cnt.groupby(["user","day"])["ip"].nunique()).reset_index()
ip_per_day_cnt=ip_per_day_cnt[["user","ip"]]
ip_per_day_cnt=ip_per_day_cnt.groupby(['user'])["ip"].agg([
    ("ip_nun_mean","mean"),
    ("ip_nun_max","max"),
    ("ip_nun_mean","mean"),
    ("ip_nun_std","std")
]).reset_index()
data=data.merge(ip_per_day_cnt, on='user', how='left')
del ip_per_day_cnt

In [71]:
# calculate the statistics of the number of different ip one may use per hour
ip_per_hour_cnt=op_df[op_df.ip!="nan"][["user","day","hour","ip"]].drop_duplicates()
ip_per_hour_cnt=pd.DataFrame(ip_per_hour_cnt.groupby(["user","day","hour"])["ip"].nunique()).reset_index()
ip_per_hour_cnt=ip_per_hour_cnt[["user","ip"]]
ip_per_hour_cnt=ip_per_hour_cnt.groupby(['user'])["ip"].agg([
    ("ip_hour_nun_max","max"),
    ("ip_hour_nun_min","min"),
    ("ip_hour_nun_mean","mean"),
    ("ip_hour_nun_std","std")
]).reset_index()
data=data.merge(ip_per_hour_cnt, on='user', how='left')
del ip_per_hour_cnt

In [72]:
# calculate the statistics of the number of different ip one may use per day per minute
ip_per_min_cnt=op_df[op_df.ip!="nan"][["user","day","hour","min","ip"]].drop_duplicates()
ip_per_min_cnt=pd.DataFrame(ip_per_min_cnt.groupby(["user","day","hour","min"])["ip"].nunique()).reset_index()
ip_per_min_cnt=ip_per_min_cnt[["user","ip"]]
ip_per_min_cnt=ip_per_min_cnt.groupby(['user'])["ip"].agg([
    ("ip_min_nun_max","max"),
    ("ip_min_nun_min","min"),
    ("ip_min_nun_mean","mean"),
    ("ip_min_nun_std","std")
]).reset_index()
data=data.merge(ip_per_min_cnt, on='user', how='left')
del ip_per_min_cnt

Done!

In [73]:
data.shape

(47782, 994)

In [74]:
# save the generate features
data.to_csv("data_1.csv",index=False)

# Feature Engineering 2

In [75]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from datetime import datetime, timedelta

from collections import Counter
import math

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from gensim.models import Word2Vec
import lightgbm as lgb

from tqdm import tqdm

import gc
import os

warnings.simplefilter('ignore')
tqdm.pandas()
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 200)

In [76]:
seed = 623

# read_data
df_train_label = pd.read_csv('train_label_new.csv', index_col=0)
df_train_base = pd.read_csv('train_base_new.csv', index_col=0)
df_train_trans = pd.read_csv('train_trans_new.csv', index_col=0)
df_train_op = pd.read_csv('train_op_new.csv')
df_train_op = df_train_op.drop(columns=['Unnamed: 0'])

df_test_base = pd.read_csv('test_base_new.csv', index_col=0)
df_test_trans = pd.read_csv('test_trans_new.csv', index_col=0)
df_test_op = pd.read_csv('test_op_new.csv', index_col=0)

df_trans = df_train_trans.append(df_test_trans)
df_trans = df_trans.reset_index(drop=True)

df_op = df_train_op.append(df_test_op)
df_op = df_op.reset_index(drop=True)

In [77]:
def parse_time(tm):
    days, _, time = tm.split(' ')
    time = time.split('.')[0]

    time = '2020-1-1 ' + time
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    time = (time + timedelta(days=int(days)))

    return time


df_trans['date'] = df_trans['tm_diff'].apply(parse_time)
df_trans['day'] = df_trans['date'].dt.day
df_trans['hour'] = df_trans['date'].dt.hour

df_op['date'] = df_op['tm_diff'].apply(parse_time)
df_op['day'] = df_op['date'].dt.day
df_op['hour'] = df_op['date'].dt.hour

In [78]:
df_trans.sort_values(['user', 'date'], inplace=True)
df_trans = df_trans.reset_index(drop=True)

df_op.sort_values(['user', 'date'], inplace=True)
df_op = df_op.reset_index(drop=True)

df_train = df_train_base.merge(df_train_label, how='left')
df_test = df_test_base

df_feature = df_train.append(df_test)

# Define embedding functions

In [79]:
os.makedirs('model', exist_ok=True)
os.makedirs('embedding', exist_ok=True)


def w2v_emb(df, f1, f2, prefix):
    emb_size = 32

    model_path = 'model/{}_w2v_{}_{}_{}.m'.format(prefix, f1, f2, emb_size)
    embedding_path = 'embedding/{}_{}_{}_{}.pkl'.format(prefix, f1, f2, emb_size)

    if os.path.exists(embedding_path):
        embedding = pd.read_pickle(embedding_path)
        return embedding

    tmp = df.groupby(f1, as_index=False)[f2].agg(
        {'{}_{}_list'.format(f1, f2): list})

    sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
    del tmp['{}_{}_list'.format(f1, f2)]
    for i in range(len(sentences)):
        sentences[i] = [str(x) for x in sentences[i]]

    if os.path.exists(model_path):
        model = Word2Vec.load(model_path)
    else:
        model = Word2Vec(sentences,
                         size=emb_size,
                         window=5,
                         min_count=5,
                         sg=0,
                         hs=1,
                         seed=seed)
        model.save(model_path)

    emb_matrix = []
    for seq in sentences:
        vec = []
        for w in seq:
            if w in model:
                vec.append(model[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * emb_size)

    df_emb = pd.DataFrame(emb_matrix)
    df_emb.columns = [
        '{}_{}_{}_emb_{}'.format(prefix, f1, f2, i) for i in range(emb_size)
    ]

    embedding = pd.concat([tmp, df_emb], axis=1)
    embedding.to_pickle(embedding_path)

    return embedding

In [80]:
def tfidf_emb(df, f1, f2, prefix):
    emb_size = 32

    df[f2] = df[f2].astype(str)
    df[f2].fillna('-1', inplace=True)
    group_df = df.groupby([f1]).apply(
        lambda x: x[f2].tolist()).reset_index()
    group_df.columns = [f1, 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=emb_size, n_iter=20, random_state=seed)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['{}_svd_tfidf_{}_{}'.format(prefix,
        f2, i) for i in range(emb_size)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

In [81]:
def countvec_emb(df, f1, f2):
    emb_size = 32

    df[f2] = df[f2].astype(str)
    df[f2].fillna('-1', inplace=True)
    group_df = df.groupby([f1]).apply(
        lambda x: x[f2].tolist()).reset_index()
    group_df.columns = [f1, 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=emb_size, n_iter=20, random_state=seed)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(
        f2, i) for i in range(emb_size)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

In [82]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

# Features from operation

In [83]:
df_op['date_diff'] = df_op.groupby('user')['date'].diff()
df_op['op_second_diff'] = df_op['date_diff'].dt.seconds
df_op['op_hour_diff'] = df_op['op_second_diff'] / 3600
df_op['op_day_diff'] = df_op['op_hour_diff'] / 24

In [84]:
for window in [15, 3, 5]:
    for col in tqdm(['op_type', 'op_mode', 'net_type', 'channel', 'hour']):
        df_temp = df_op[df_op['day'] > 15 - window][['user', col]].copy()
        df_temp['tmp'] = 1
        df_temp = df_temp.pivot_table(index='user', columns=col,
                                      values='tmp', aggfunc=np.sum).reset_index().fillna(0)
        df_temp.columns = [c if c == 'user' else 'op_{}_{}_count_{}d'.format(
            col, c, window) for c in df_temp.columns]
        df_feature = df_feature.merge(df_temp, how='left')

100%|██████████| 5/5 [00:03<00:00,  1.32it/s]


In [85]:
for col in tqdm(['op_type', 'op_mode', 'net_type', 'channel']):
    df_temp = df_op[['user', 'hour', col]].copy()
    df_temp = df_temp.pivot_table(index='user', columns=col,
                                  values='hour', aggfunc=['mean', 'std', 'max', 'min']).fillna(0)
    df_temp.columns = ['op_{}_{}_hour_{}'.format(col, f[1], f[0]) for f in df_temp.columns]
    df_temp.reset_index(inplace=True)
    df_temp.rename({'index': 'user'}, inplace=True, axis=1)
    df_feature = df_feature.merge(df_temp, how='left')

100%|██████████| 4/4 [00:11<00:00,  2.85s/it]


In [86]:
df_temp = df_op.groupby(['user', 'op_device']).size().reset_index()
df_temp.drop([0], axis=1, inplace=True)
df_temp = df_temp.sort_values(
    by=['user', 'op_device'], ascending=['asc', 'asc'])
df_temp.drop_duplicates('user', keep='last', inplace=True)
df_feature = df_feature.merge(df_temp, how='left')

In [87]:
for f in ['hour', 'day', 'op_second_diff']:
    df_temp = df_op.groupby('user')[f].agg([
        ('op_{}_mean'.format(f), 'mean'), 
        ('op_{}_std'.format(f), 'std'), 
        ('op_{}_max'.format(f), 'max'),
        ('op_{}_min'.format(f), 'min')
    ]).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')

# Features from transaction

In [88]:
df_trans['date_diff'] = df_trans.groupby('user')['date'].diff()
df_trans['trans_second_diff'] = df_trans['date_diff'].dt.seconds
df_trans['trans_hour_diff'] = df_trans['trans_second_diff'] / 3600
df_trans['trans_day_diff'] = df_trans['trans_hour_diff'] / 24

df_trans['amount'] = np.log1p(df_trans['amount'])

In [89]:
for col in tqdm(['platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2', 'hour']):
    df_temp = df_trans.pivot_table(
        index='user', columns=col, values='amount', aggfunc=['sum', 'mean', 'max', 'min', 'std', 'median']).fillna(0)
    df_temp.columns = ['trans_{}_{}_amount_{}'.format(col, f[1], f[0]) for f in df_temp.columns]
    df_temp.reset_index(inplace=True)
    df_temp.rename({'index': 'user'}, inplace=True, axis=1)

    df_feature = df_feature.merge(df_temp, how='left')

100%|██████████| 6/6 [00:09<00:00,  1.59s/it]


In [90]:
for window in tqdm([31, 1, 3, 5, 7, 10, 15]):
    df_temp = df_trans[df_trans['day'] > 31-window].groupby('user')['amount'].agg([
        ('trans_amount_mean_{}d'.format(window), 'mean'),
        ('trans_amount_std_{}d'.format(window), 'std'),
        ('trans_amount_max_{}d'.format(window), 'max'),
        ('trans_amount_min_{}d'.format(window), 'min'),
        ('trans_amount_sum_{}d'.format(window), 'sum'),
    ]).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')

100%|██████████| 7/7 [00:03<00:00,  2.04it/s]


In [91]:
for window in [3, 5, 10]:
    for col in ['type1', 'type2']:
        df_temp = df_trans[df_trans['day'] > 31 - window].pivot_table(
            index='user', columns=col, values='amount', aggfunc=['sum']).fillna(0)
        df_temp.columns = ['trans_{}_{}_amount_{}_{}d'.format(col, f[1], f[0], window) for f in df_temp.columns]
        df_temp.reset_index(inplace=True)
        df_temp.rename({'index': 'user'}, inplace=True, axis=1)
        
        df_feature = df_feature.merge(df_temp, how='left')

In [92]:
for f in ['ip', 'ip_3']:    
    df_temp = df_trans.groupby(['user'])[f].agg([
        ('trans_{}_count'.format(f), 'count')
    ]).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')

In [93]:
for f in ['hour', 'trans_day_diff']:
    df_temp = df_trans.groupby('user')[f].agg([
        ('trans_{}_mean'.format(f), 'mean'), 
        ('trans_{}_std'.format(f), 'std'),
    ]).reset_index()
    df_feature = df_feature.merge(df_temp, how='left')

# Features from base

In [94]:
for f in [
        'balance', 'balance_avg', 'balance1', 'balance1_avg', 'balance2',
        'balance2_avg', 'product1_amount', 'product2_amount',
        'product3_amount', 'product4_amount', 'product5_amount', 'product6_amount'
]:
    df_feature[f] = df_feature[f].apply(lambda x: int(
        x.split(' ')[1]) if type(x) != float else np.NaN)

In [95]:
cate_features = ['sex', 'provider', 'level', 'verified', 'regist_type', 'agreement1', 'agreement2', 'agreement3', 'agreement4', 'province', 'city', 'service3', 
                 'service3_level']

for f1 in tqdm(cate_features):
    for f2 in cate_features:
        df_feature['{}_{}'.format(f1, f2)] = df_feature[f1] + '_' + df_feature[f2]
        
for f in tqdm(cate_features):
    df_feature['{}_cnt'.format(f)] = df_feature.groupby([f])['user'].transform('count')

100%|██████████| 13/13 [00:00<00:00, 185.66it/s]


In [96]:
dense_features = ['age', 'using_time', 'card_a_cnt', 'card_b_cnt', 'card_c_cnt', 'card_d_cnt', 'op1_cnt', 'op2_cnt', 'service1_cnt', 'service1_amt', 'service2_cnt', 
                  'agreement_total', 'acc_count', 'login_cnt_period1', 'login_cnt_period2', 'ip_cnt', 'login_cnt_avg', 'login_days_cnt', 'balance', 'balance_avg', 
                  'balance1', 'balance1_avg', 'balance2', 'balance2_avg', 'product1_amount', 'product2_amount', 'product3_amount', 'product4_amount', 'product5_amount',
                 'product6_amount', 'product7_cnt', 'product7_fail_cnt']

min_max = MinMaxScaler()
df_feature[dense_features] = min_max.fit_transform(df_feature[dense_features].values)

for f1 in tqdm(dense_features):
    for f2 in dense_features:
        if f1 != f2:
            df_feature['{}_add_{}'.format(f1, f2)] = df_feature[f1] + df_feature[f2]

100%|██████████| 32/32 [00:08<00:00,  3.71it/s]


In [97]:
df_feature['product7_fail_ratio'] = df_feature[
    'product7_fail_cnt'] / df_feature['product7_cnt']
df_feature['card_cnt'] = df_feature['card_a_cnt'] + df_feature[
    'card_b_cnt'] + df_feature['card_c_cnt'] + df_feature['card_d_cnt']

df_feature['acc_card_ratio'] = df_feature['acc_count'] / df_feature['card_cnt']
df_feature['login_cnt'] = df_feature['login_cnt_period1'] + \
    df_feature['login_cnt_period2']

df_feature['login_cnt_period2_login_cnt_ratio'] = df_feature['login_cnt_period2'] / \
    df_feature['login_cnt']
df_feature['login_cnt_period1_login_cnt_ratio'] = df_feature['login_cnt_period1'] / \
    df_feature['login_cnt']

df_feature['using_time_op2_cnt_ratio'] = df_feature['using_time'] / \
    df_feature['op2_cnt']
df_feature['using_time_op1_cnt_ratio'] = df_feature['using_time'] / \
    df_feature['op1_cnt']

In [98]:
# 欺诈率
def stat(df, df_merge, group_by, agg):
    group = df.groupby(group_by).agg(agg)

    columns = []
    for on, methods in agg.items():
        for method in methods:
            columns.append('{}_{}_{}'.format('_'.join(group_by), on, method))
    group.columns = columns
    group.reset_index(inplace=True)
    df_merge = df_merge.merge(group, on=group_by, how='left')

    del (group)
    gc.collect()
    return df_merge


def statis_feat(df_know, df_unknow):
    df_unknow = stat(df_know, df_unknow, ['province'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['city'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, [
                     'city', 'level'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['op_device'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, [
                     'age', 'op_device'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['using_time'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, [
                     'city', 'op_device'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, ['age', 'city'], {'label': ['mean']})
    df_unknow = stat(df_know, df_unknow, [
                     'op_device', 'level'], {'label': ['mean']})

    return df_unknow

In [99]:
df_train = df_feature[~df_feature['label'].isnull()]
df_train = df_train.reset_index(drop=True)
df_test = df_feature[df_feature['label'].isnull()]

In [100]:
df_stas_feat = None
kf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(df_train, df_train['label']):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]

    df_fold_val = statis_feat(df_fold_train, df_fold_val)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)

    del (df_fold_train)
    del (df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test)
df_feature = pd.concat([df_stas_feat, df_test], axis=0)
df_feature = df_feature.reset_index(drop=True)

del (df_stas_feat)
del (df_train)
del (df_test)
gc.collect()

0

In [101]:
for f in tqdm(cate_features):
    for f2 in dense_features:
        df_feature['{}_{}_mean'.format(f, f2)] = df_feature.groupby([f])[f2].transform('mean')

100%|██████████| 13/13 [00:04<00:00,  2.69it/s]


In [102]:
df_feature.shape

(47782, 4253)

In [103]:
# save the generate features
df_feature.to_csv("data_2.csv",index=False)